In [1]:
import os
import mne
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, permutation_test

# Functions

In [2]:
def calc_pearson_corr(in_ear_data, scalp_data):
    return pearsonr(in_ear_data.ravel(), scalp_data.ravel())[0]

# Fetch Data

In [3]:
main_path = './Dataset'

In [4]:
sbj_records = []
for sub in os.listdir(main_path):
    tmp = []
    for file in os.listdir(f'./Dataset/{sub}'):
        if file.split(sep='.')[1] == 'dat':
            tmp.append(mne.io.read_raw(f'./Dataset/{sub}/{file}', preload=True))
    sbj_records.append(mne.concatenate_raws(tmp))
        

Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 981199  =      0.000 ...   981.199 secs...
Event file found. Extracting Annotations from C:\Users\DFMRendering\Desktop\python\in-ear\Dataset\subject1\motor1.ceo...
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 984999  =      0.000 ...   984.999 secs...
Event file found. Extracting Annotations from C:\Users\DFMRendering\Desktop\python\in-ear\Dataset\subject1\motor2.ceo...
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 1022899  =      0.000 ...  1022.899 secs...
Event file found. Extracting Annotations from C:\Users\DFMRendering\Desktop\python\in-ear\Dataset\subject1\motor3.ceo...
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 2031599  =      0.000 ...  2031.599 secs...
Event file found. Extracting Annotations from C:\Users\DFMRendering\Desktop\python\in-ear\Dataset\subject2\motor.ceo...
Leaving device<->head transform as None (no

In [5]:
sbj_records[0]

Measurement date,"October 08, 2019 15:18:19 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,125 points
Good channels,"122 EEG, 10 misc"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.00 Hz
Lowpass,500.00 Hz


# Preprocessing

1. filter data with a band pass between **0.5** and **100**hz
2. get the difference of channel correlation and psd of (for evaluation):
   - each ear channel (all channels)
   - adjacent scalp channel 
   - motor related channels 
3. downsampling data from **1000** Hz to **250** Hz
4. filter downsampled data using with **4 - 38 Hz** FIR filter
5. drop **FP1** channel of subject 4 , beacuase it was broken
6. **interpolate** other scalp channels and append instead of subject 4's FP1 that has been droped (using Blue channels)
7. epoch data by **[-2, 6]** window (what does it mean?)
8. reject epochs which has data point higher than **6$\sigma$** for single channel and **2$\sigma$** for all channels
9. get psd difference from :
    - all inear channels
    - scalp ajacent (**T8,T7,T9,T10**) - Blue channels
    - motor task - Red channels
10. The mean power of Theta(4-8 Hz), alpha(8-13 Hz), Beta(13-30 Hz), Low gamma(30-38 Hz) were calculated by 8s (-2-6)(?) epochs and get the average of all epochs
11. the mean power of Left or Right in ear biopotensial (Mean of signal's power) devided by T9/T10 of scalp channels.
12. in a similar way to part 11 we get devided by C3/C4 of motor channels
13. to calculate channeles correlation we have used pearson correlation for each 8 ear channels with 122 scalp channels for each subject
14. apply a permution test between in-ear channeles and motor channels (C3 and C4)
15. 5000 times test(?)

**filter with band pass between 0.5 and 100 Hz**
 (subsection 2.1)

In [6]:
for sample in sbj_records:
    sample.filter(0.5, 100)

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.4s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    2.8s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 6601 samples (6.601 s)



...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.5s


Filtering raw data in 4 contiguous segments
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.7s


Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.7s


**Downsampling from 1000 to 250**

In [7]:
for data in sbj_records:
    data.resample(250)

**filter with a 4-38 Hz FIR filter**

In [8]:
for data in sbj_records:
    data.filter(l_freq=4, h_freq=38, method='fir')

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.3s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.7s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.3s


Filtering raw data in 4 contiguous segments
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.4s


Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.4s


**Mark FP1 as bad channel and interpolate using other channels**

In [9]:
bad_channels = ['FP1']

In [10]:
sbj_records[3].info['bads'] = bad_channels

In [11]:
sbj_records[3].interpolate_bads(reset_bads=True)

Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 96.5 mm
Computing interpolation matrix from 121 sensor positions
Interpolating 1 sensors


Measurement date,"November 09, 2019 17:47:34 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,125 points
Good channels,"122 EEG, 10 misc"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,4.00 Hz
Lowpass,38.00 Hz


**re-referencing using averaging method**

In [12]:
for sbj in sbj_records:
    sbj.set_eeg_reference(ref_channels='average', projection=True)

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projecti

**apply epochs which range are [-2, 6]**

In [13]:
epochs = []
for sbj in sbj_records:
    # events = mne.make_fixed_length_events(sbj, duration=8.0)
    epochs.append(mne.Epochs(sbj, tmin=-2, tmax=6, baseline=None, detrend=1, event_repeated='merge'))

Used Annotations descriptions: ['1', '2', '700010']
Not setting metadata
242 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Used Annotations descriptions: ['1', '2']
Not setting metadata
160 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Used Annotations descriptions: ['1', '2', '800000', '800001']
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
163 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Used Annotations descriptions: ['1', '2', '800000', '800001']
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
81 matching events found
No baseline correction applied
Created an SSP oper

**delete epochs which are greater than 2$\sigma$ of all channels**

In [14]:
clean_epochs = []
for sbj_epoch in epochs:
    all_data = sbj_epoch.get_data()
    threshold = 2 * np.std(all_data)
    reject_criteria = dict(eeg=threshold)
    sbj_epoch.drop_bad(reject=reject_criteria)
    clean_epochs.append(sbj_epoch)

Using data from preloaded Raw for 242 events and 2001 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 242 events and 2001 original time points ...
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FT9']
2 bad epochs dropped
Using data from preloaded Raw for 160 events and 2001 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 160 events and 2001 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 163 events and 2001 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 163 events and 2001 original time points ...
    Rejecting  epoch based on EEG : ['POO12h']
    Rejecting  epoch based on EEG : ['POO12h']
    Rejecting  epoch based on EEG : ['POO12h']
    Rejecting  epoch based on EEG : ['POO12h']
    Rejecting  epoch based on EEG : ['POO12h']
    Rejecting  epoch based on EEG : ['P12']
    Rejecting  epoch based on EEG : ['P12']
7 bad epoc

In [15]:
clean_epochs[0].info

Measurement date,"October 08, 2019 15:18:19 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,125 points
Good channels,"122 EEG, 10 misc"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,4.00 Hz
Lowpass,38.00 Hz


**apply ICA filter on each epoch**

In [16]:
ica = mne.preprocessing.ICA(n_components=20)

In [17]:
for sbj_epoch in clean_epochs:
    sbj_epoch = ica.fit(sbj_epoch)

Fitting ICA to data using 122 channels (please be patient, this may take a while)
Using data from preloaded Raw for 240 events and 2001 original time points ...
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 20 components
Using data from preloaded Raw for 240 events and 2001 original time points ...
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 24.5s.
Fitting ICA to data using 122 channels (please be patient, this may take a while)
Using data from preloaded Raw for 160 events and 2001 original time points ...
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 20 components
Using data from preloaded Raw for 160 events and 2001 original time points ...
    Applying projection operator with 1 vector (pre-whit

**select channels**

In [18]:
selected = []
for sbj_epoch in clean_epochs:
    sbj_epoch.load_data() 
    sbj_epoch_copy = sbj_epoch.copy()
    selected.append(sbj_epoch_copy.pick_channels(['T9','T10','C3','C4','LF','LB','LOU','LOD','RF','RB','ROU','ROD']))

Using data from preloaded Raw for 240 events and 2001 original time points ...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Using data from preloaded Raw for 160 events and 2001 original time points ...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Using data from preloaded Raw for 156 events and 2001 original time points ...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Using data from preloaded Raw for 79 events and 2001 original time points ...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Using data from preloaded Raw for 235 events and 2001 original time points ...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Using data from preloaded Raw for 240 events and 2001 original time points ...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


**calculate the PSD of each epochs**

In [19]:
ear_scalp = {
    'theta': [],
    'alpha': [],
    'batha': [], 
    'low gamma':[]
}
ear_motor = {
    'theta': [],
    'alpha': [],
    'batha': [], 
    'low gamma':[]
}

In [20]:
freqs = [(4, 8), (8, 13), (13, 30), (30, 38)]
keys = list(ear_scalp.keys())
for fr in range(4):
    f_min, f_max = freqs[fr]
    for sbj in selected:
        psd_t9 = mne.time_frequency.psd_array_welch(sbj.get_data(['T9']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]
        psd_t10 = mne.time_frequency.psd_array_welch(sbj.get_data(['T10']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]
        psd_c3 = mne.time_frequency.psd_array_welch(sbj.get_data(['C3']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]
        psd_c4 = mne.time_frequency.psd_array_welch(sbj.get_data(['C4']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]
        # scalp - right
        ear_scalp[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['LF']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_t9)                  
        ear_scalp[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['LB']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_t9)            
        ear_scalp[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['LOU']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_t9)            
        ear_scalp[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['LOD']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_t9)    
        # scalp - left
        ear_scalp[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['RF']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_t10)                  
        ear_scalp[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['RB']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_t10)            
        ear_scalp[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['ROU']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_t10)            
        ear_scalp[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['ROD']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_t10)  
        # motor - right
        ear_motor[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['LF']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_c3)                  
        ear_motor[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['LB']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_c3)            
        ear_motor[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['LOU']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_c3)            
        ear_motor[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['LOD']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_c3) 
        # motor - left
        ear_motor[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['RF']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_c4)                  
        ear_motor[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['RB']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_c4)            
        ear_motor[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['ROU']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_c4)            
        ear_motor[keys[fr]].append(mne.time_frequency.psd_array_welch(sbj.get_data(['ROD']), fmin=f_min, fmax=f_max, sfreq=sbj.info['sfreq'])[0]/psd_c4) 


Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective wind

calculate pearson correlation between **in-ear** and **scalp** and **motor** channels

In [21]:
ear_channels = ['LF','LB','LOU','LOD','RF','RB','ROU','ROD']
other = ['T9', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'T10','LF','LB','LOU','LOD','RF','RB','ROU','ROD']
correlations = []
for sbj in clean_epochs:
    tmp = []
    for ear in ear_channels:
        ear_data = sbj.get_data(picks=[ear])
        for ch in other:
            scalp_data = sbj.get_data(picks=[ch])
            corr, _ = pearsonr(ear_data, scalp_data)
            tmp.append(corr)
    correlations.append(np.array(tmp))
correlations = np.array(correlations)

In [22]:
correlations.shape

(6, 152, 1, 2001)

**Permution test**

In [23]:
motor_channels = ['C3', 'C4']

In [24]:
num_permution = 10

In [25]:
permuted_correlations = np.zeros((6, len(ear_channels), len(motor_channels), num_permution))

In [26]:
permuted_correlations.shape

(6, 8, 2, 10)

In [27]:
# subject = 0
# for epoch in clean_epochs:
#     for permution in range(num_permution):
#         epoch_copy = epoch.copy()
#         epoch_copy._data = np.random.permutation(epoch_copy.get_data())
#         for i, in_ear_ch in enumerate(ear_channels):
#             for j, motor_ch in enumerate(motor_channels):
#                 in_ear_data = epoch_copy.get_data(picks=[in_ear_ch]).ravel()
#                 motor_data = epoch_copy.get_data(picks=[motor_ch]).ravel()
#                 permuted_corr, _ = pearsonr(in_ear_data, motor_data)
#                 permuted_correlations[subject, i, j, permution] = permuted_corr
#     subject += 1

In [28]:
p_values = {}
observed_correlations = {}

In [29]:
epoch_copy = clean_epochs[0].copy()
for i, in_ear_ch in enumerate(ear_channels):
    p_values[in_ear_ch] = {}
    observed_correlations[in_ear_ch] = {}
    for j, motor_ch in enumerate(motor_channels):
        in_ear_data = epoch_copy.get_data(picks=[in_ear_ch])
        motor_data = epoch_copy.get_data(picks=[motor_ch])
        # Calculate observed correlation
        observed_corr = pearsonr(in_ear_data.ravel(), motor_data.ravel())[0]
        observed_correlations[in_ear_ch][motor_ch] = observed_corr
        # Perform permutation test
        perm_test_result = permutation_test(
            data=(in_ear_data, motor_data),
            statistic=calc_pearson_corr,
            vectorized=False,
            n_resamples=100,
            alternative='two-sided'
        )
        # Store p-value
        p_values[in_ear_ch][motor_ch] = perm_test_result.pvalue

In [31]:
observed_correlations

{'LF': {'C3': -0.0007065683815221342, 'C4': -0.002266194561674083},
 'LB': {'C3': -0.0009119096740704763, 'C4': -0.0027244027856115657},
 'LOU': {'C3': -0.000828584626615553, 'C4': -0.0031608294525883895},
 'LOD': {'C3': -0.0009750973627862487, 'C4': -0.0029421631121759138},
 'RF': {'C3': -0.0022153076073480288, 'C4': -0.0016688612067300645},
 'RB': {'C3': -0.006667837713586108, 'C4': -0.004834866134275631},
 'ROU': {'C3': -0.002492121849426746, 'C4': -0.0015508879261253},
 'ROD': {'C3': -0.0007661951500670377, 'C4': -0.0005690316698566038}}

In [32]:
p_values

{'LF': {'C3': array([[0.79207921, 0.67326733, 0.53465347, ..., 0.23762376, 0.25742574,
          0.27722772]]),
  'C4': array([[0.81188119, 0.65346535, 0.35643564, ..., 0.65346535, 0.67326733,
          0.89108911]])},
 'LB': {'C3': array([[0.95049505, 0.83168317, 0.59405941, ..., 0.23762376, 0.31683168,
          0.31683168]]),
  'C4': array([[0.73267327, 0.47524752, 0.27722772, ..., 0.4950495 , 0.53465347,
          0.81188119]])},
 'LOU': {'C3': array([[0.97029703, 0.81188119, 0.57425743, ..., 0.31683168, 0.31683168,
          0.33663366]]),
  'C4': array([[0.4950495 , 0.35643564, 0.1980198 , ..., 0.45544554, 0.4950495 ,
          0.83168317]])},
 'LOD': {'C3': array([[0.73267327, 0.57425743, 0.3960396 , ..., 0.41584158, 0.53465347,
          0.57425743]]),
  'C4': array([[0.53465347, 0.41584158, 0.35643564, ..., 0.45544554, 0.47524752,
          0.73267327]])},
 'RF': {'C3': array([[0.81188119, 0.75247525, 0.61386139, ..., 0.21782178, 0.23762376,
          0.2970297 ]]),
  'C4': ar